# BB84 Quantum Key Distribution Simulation
This notebook demonstrates the BB84 protocol for Quantum Key Distribution (QKD) using Qiskit. It explains the steps involved in the protocol and shows how Qiskit can be used to simulate this process.

## Introduction
The BB84 protocol is one of the most well-known quantum cryptographic protocols, introduced by Charles Bennett and Gilles Brassard in 1984. It allows two parties, Alice and Bob, to share a cryptographic key securely.

In [34]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
import numpy as np

### Step 1: Alice Prepares the Qubits
Alice generates a random string of bits and chooses random bases (rectilinear or diagonal) for each qubit. Based on these, Alice prepares the qubits.

In [35]:
def prepare_qubits(bits, bases):
    qubits = []
    for bit, basis in zip(bits, bases):
        qc = QuantumCircuit(1, 1)
        if bit == 1:
            qc.x(0)
        if basis == 1:
            qc.h(0)
        qubits.append(qc)
    return qubits

### Step 2: Bob Measures the Qubits
Bob randomly chooses a basis for each qubit and measures it.

In [36]:
def measure_qubits(qubits, bases):
    measurements = []
    simulator = AerSimulator()

    for qc, basis in zip(qubits, bases):
        if qc.num_qubits < 1 or qc.num_clbits < 1:
            raise ValueError("QuantumCircuit muss mindestens 1 Qubit und 1 klassisches Bit haben.")
        
        if basis == 1:
            qc.h(0)
        
        qc.measure(0, 0)
        
        compiled_circuit = transpile(qc, simulator)
        
        sim_result = simulator.run(compiled_circuit, shots=1).result()
        counts = sim_result.get_counts()
        
        measured_bit = max(counts, key=counts.get)
        measurements.append(int(measured_bit))
    
    return measurements

### Simulation Parameters
We set the number of qubits, generate random bit strings, and run the simulation.

In [37]:
np.random.seed(42)
n = 10  # Number of qubits
alice_bits = np.random.randint(2, size=n)
alice_bases = np.random.randint(2, size=n)
bob_bases = np.random.randint(2, size=n)

alice_qubits = prepare_qubits(alice_bits, alice_bases)
bob_measurements =  measure_qubits(alice_qubits, bob_bases)

### Key Comparison
Alice and Bob compare their bases to establish a shared key.

In [38]:
key_agreement = [ab == bb for ab, bb in zip(alice_bases, bob_bases)]
shared_key = [bm for ka, bm in zip(key_agreement, bob_measurements) if ka]

print('Alices Bits:     ', alice_bits)
print('Alices Bases:    ', alice_bases)
print('Bobs Bases:      ', bob_bases)
print('Bobs Measurements:', bob_measurements)
print('Shared Key:       ', shared_key)

Alices Bits:      [0 1 0 0 0 1 0 0 0 1]
Alices Bases:     [0 0 0 0 1 0 1 1 1 0]
Bobs Bases:       [1 0 1 1 1 1 1 1 1 1]
Bobs Measurements: [1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
Shared Key:        [1, 0, 0, 0, 0]
